<a href="https://colab.research.google.com/github/shekarramaswamy4/tf-pytorch-sandbox/blob/master/pytorch_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Purpose: refreshing memory of pytorch basics
from __future__ import print_function
import torch

In [4]:
x = torch.empty(5, 3)
x = torch.rand(5, 3)
x = torch.zeros(5, 3, dtype=torch.long)
x = x.new_ones(5, 3, dtype=torch.double)
print(x.size())
y = torch.rand(5, 3)
z = x + y
print(z)

y.add_(x) # adds in place. any operation that mutates in place is post-fixed with _

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
torch.Size([5, 3])
tensor([[1.7063, 1.1468, 1.7186],
        [1.0620, 1.7752, 1.3976],
        [1.3856, 1.3264, 1.4275],
        [1.8696, 1.3355, 1.4415],
        [1.7525, 1.4986, 1.3373]], dtype=torch.float64)


In [8]:
print(z[:2, 1])

tensor([1.1468, 1.7752], dtype=torch.float64)


In [11]:
x = torch.randn(5, 4)
y = x.view(4, 5)
print(x)
print(y)

tensor([[-0.2273,  2.1592,  0.4239, -0.0546],
        [-0.3259, -1.2157, -0.0155, -1.3147],
        [-0.7765, -0.0306, -0.6572, -1.5288],
        [-0.4717, -0.5740,  2.1837,  0.4893],
        [ 0.5890, -0.2862, -1.0189,  0.7803]])
tensor([[-0.2273,  2.1592,  0.4239, -0.0546, -0.3259],
        [-1.2157, -0.0155, -1.3147, -0.7765, -0.0306],
        [-0.6572, -1.5288, -0.4717, -0.5740,  2.1837],
        [ 0.4893,  0.5890, -0.2862, -1.0189,  0.7803]])


In [15]:
# torch <=> numpy
a = torch.ones(3)
b = a.numpy()
b = b + 1
print(b)

c = torch.from_numpy(b)
a.add_(2)
print(a)
print(b)
print(c)

[2. 2. 2.]
tensor([3., 3., 3.])
[2. 2. 2.]
tensor([2., 2., 2.])


In [0]:
# cuda
if torch.cuda.is_available():
  device = torch.device("cuda")
  y = torch.ones(3, device=device)
  x = torch.ones(3)
  z = x + y # this shouldn't work
  print(z)
  print(z.to("cpu", torch.double))

In [19]:
# Autograd
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()
print(z, out)
out.backward()
print(x.grad)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [22]:
# torch.nn
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)
print(net.parameters())

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
<generator object Module.parameters at 0x7f4061eb9d58>


In [24]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()
loss = criterion(out, target)
print(loss)

tensor(1.9583, grad_fn=<MseLossBackward>)


In [0]:
# optimizers
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.01)
# output, get loss, loss.backward(), optimizer.step()